# Chess-Bot

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import chessbot as cb

## Dataset

In [ ]:
cb.dataset.create()

In [ ]:
import numpy as np
import pandas as pd

win = pd.read_parquet("./data/win.parquet", engine="fastparquet")
print(win.head())

print(np.unique(win.iloc[0]["bitboard"], return_counts=True))

In [ ]:
cb.dataset.create()